In [1]:
import os

os.environ

environ{'SHELL': '/bin/bash',
        'NVM_RC_VERSION': '',
        'VSCODE_VERBOSE_LOGGING': 'true',
        'PYTHONUNBUFFERED': '1',
        'CONDA_EXE': '/opt/conda/bin/conda',
        '_CE_M': '',
        'VSCODE_LOG_STACK': 'false',
        'HOSTNAME': '73e230fca95e',
        'SSH_AUTH_SOCK': '/tmp/vscode-ssh-auth-33595b77d2353fc45228d904dbd09f0217a36354.sock',
        'VSCODE_INJECT_NODE_MODULE_LOOKUP_PATH': '/vscode/vscode-server/bin/linux-x64/6cba118ac49a1b88332f312a8f67186f7f3c1643/remote/node_modules',
        'REMOTE_CONTAINERS_IPC': '/tmp/vscode-remote-containers-ipc-33595b77d2353fc45228d904dbd09f0217a36354.sock',
        'VSCODE_PIPE_LOGGING': 'true',
        'VSCODE_AMD_ENTRYPOINT': 'vs/server/remoteExtensionHostProcess',
        'PWD': '/vscode/vscode-server/bin/linux-x64/6cba118ac49a1b88332f312a8f67186f7f3c1643',
        'PIPX_BIN_DIR': '/usr/local/py-utils/bin',
        'CONDA_PREFIX': '/opt/conda',
        'TZ': 'Asia/Ulaanbaatar',
        'HOME': '/home/vscode',
    